___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*ip38LhwnKDrCjFTNgPFacQ.jpeg" width="350px" height="180px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
#### <font color= #2E9AFE> `Martes y Viernes (Videoconferencia) de 13:00 - 15:00 hrs`</font>
- <Strong> Sara Eugenia Rodríguez </Strong>
- <Strong> Año </Strong>: 2025
- <Strong> Email: </Strong>  <font color="blue"> `cd682324@iteso.mx` </font>
___

<p style="text-align:right;"> Imagen recuperada de: https://miro.medium.com/v2/resize:fit:1100/format:webp/1*ip38LhwnKDrCjFTNgPFacQ.jpeg</p>


### <font color= #2E9AFE> Tema: Reducción de dimensionalidad</font>

La **maldición de la dimensionalidad** se refiere a los problemas que ocurren cuando los datos tienen demasiadas características o dimensiones.

A medida que aumenta el número de dimensiones:

- Los datos se vuelven dispersos.
- Se vuelve más difícil encontrar patrones o relaciones a medida que se agrega más y más ruido a los datos.
- Los algoritmos se vuelven más lentos y menos precisos.

**Ejemplo:**

Digamos que queremos predecir el precio de una casa. Al inicio tenemos tres variables:
1. Tamaño de la casa
2. Número de cuartos
3. Ubicación

Con esto es fácil encontrar patrones y hacer buenas predicciones. 

Podríamos empezar a pensar en más caracteríasticas que pudieran ayudarnos a ser más precisos en nuestras predicciones, como:

- Color de las paredes
- Tipo de cortinas
- Forma de las agarraderas para abrir las puertas
- Marca del fregadero de la cocina
- Tipo de piso
- Tipos de focos
- Etc...

Ahora hemos agregado más de 30 características... Por sentido común, la mayoría de estos realmente no afectan el precio, como la marca del fregadero de la cocina, el tipo de cortinas, etc.

Entonces, no siempre es necesario que agregando más características siempre obtengamos mejores resultados.

**Qué pasa cuando más características son agregadas?**

- Es más difícil encontrar patrones que valgan la pena
- El modelo se hace más lento y puede dar peores resultados
- Necesitamos muchísima más data para entrenar el modelo

### Esto es la maldición de la dimensionalidad: demasidas características pueden perjudicar nuestros resultados

**Formas de quitar la maldición de la dimensionalidad**

1. Selección de Variables
2. Reducción de Dimensionalidad = Extracción de variables

La selección de características es diferente a la reducción de dimensionalidad. 

Ambos métodos buscan reducir la cantidad de atributos en el conjunto de datos, pero un método de reducción de dimensionalidad lo hace creando nuevas combinaciones de atributos, donde los métodos de selección de características incluyen y excluyen atributos presentes en los datos sin cambiarlos.

Ejemplos de métodos de reducción de dimensionalidad incluyen: Análisis de componentes principales y Descomposición de valores singulares.

### Principant Component Analysis 

El PCA usa álgebra lineal para transformar el conjunto de datos en una forma comprimida.

**El objetivo de PCA**

Identificar patrones, correlaciones entre variables y luego comprimir las variables en sus características más importantes para que los datos se simplifiquen sin perder información importante.

**Cómo determinar si el PCA es el mejor método para ti**

1. ¿Quieres reducir el número de variables, pero no puedes eliminar ninguna por completo?
2. Tienes demasiadas variables correlacionadas?
3. Tus datos son lineales
4. ¿Te sientes cómodo perdiendo algo de interpretabilidad de tus variables independientes?
  
Si respondió “sí” a las tres preguntas... entonces PCA es para ti.

Por cierto...el PCA se beneficia de la estandarización de los datos

In [ ]:
#Importar librerías
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer

In [ ]:
#cargar datos
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')
df = pd.concat([X,y], axis=1)

In [ ]:
df.shape

In [ ]:
df.head()

### Procesamiento de datos

Antes de realizar PCA, es fundamental estandarizar los datos. Esto significa centrar los datos restando la media y escalarlos dividiéndolos por la desviación estándar. La estandarización garantiza que todas las características tengan la misma importancia en el análisis.

In [ ]:
#Escalar datos 
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_scaled = sc.fit_transform(X)

### Eligiendo el número de componentes principales

In [ ]:
from sklearn.decomposition import PCA
 
#inicializar el objeto
pca = PCA()
#aplicarlo a los datos
pca_fit = pca.fit_transform(x_scaled)
 

In [ ]:
#Variables transformadas (rotadas)
pd.DataFrame(pca_fit, columns=X.columns)

In [ ]:
# Graficar la varianza acumulativa de cada componente
plt.figure(figsize = (15, 6))
components = np.arange(1, 31, step=1) 
variance = np.cumsum(pca.explained_variance_ratio_)
plt.ylim(0.0,1.1)
plt.plot(components, variance, marker='o', linestyle='--', color='green')
plt.xlabel('Number of Components')
plt.xticks(np.arange(1, 14, step=1))
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')
plt.axhline(y=0.90, color='r', linestyle='-')
plt.text(0.5, 0.85, '90% variance threshold', color = 'red', fontsize=16)
plt.text(25, 0.85, "Components needed: "+str(np.where(np.cumsum(pca.explained_variance_ratio_)>=0.9)[0][0]), color = "red", fontsize=16)
plt.show()

El gráfico anterior contiene la varianza acumulada como una proporción en el eje Y y el número de componentes en el eje X.

Utilizando un umbral de variación del 90%, el gráfico anterior nos ayuda a determinar cuántos componentes debemos conservar de nuestro conjunto de datos para que siga teniendo sentido para nosotros en cualquier modelado posterior.

Tenga en cuenta que aquí elegimos el 90% como umbral de variación, pero esta no es una regla de oro. El data scientist elige este umbral de variación.

Entonces, podemos ver en este gráfico que necesitamos 6 componentes para retener el 90% de la variabilidad (información) en nuestros datos.

### Aplicando el PCA

In [ ]:
from sklearn.decomposition import PCA
 
#inicializar el objeto
pca = PCA(n_components=6)
#aplicarlo a los datos
pca_fit = pca.fit_transform(x_scaled)
datos_pca = pd.DataFrame(pca_fit, columns=['PC1','PC2','PC3','PC4','PC5','PC6'])
datos_pca['y'] = y

In [ ]:
datos_pca

In [ ]:
#sin PCA
##Aplicacion de la regresion logística con datos originales
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#inicializar el objeto
logreg = LogisticRegression()
#aplicar la regresión logística a los datos
logreg.fit(x_scaled, y)

#predicciones
y_predict = logreg.predict(x_scaled)
accuracy_score(y,y_predict)


In [ ]:
##Aplicacion de la regresion logística con datos después de aplicar el PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(np.array(datos_pca[['PC1','PC2','PC3','PC4','PC5','PC6']]), y)

y_predict = logreg.predict(datos_pca[['PC1','PC2','PC3','PC4','PC5','PC6']])
accuracy_score(y,y_predict)


No se aumenta el accuracy con el PCA (fenómeno normal) pero sí reducimos la dimensionalidad de los datos, por lo tanto es menos carga computacional. Con menos información, el clasificador sigue dando muy buenos resultados. 


### ¿Qué algoritmos se benefician del PCA?

1. Algoritmos lineales. Ya que ayuda a reducir la multicolinealidad
    - Regresión logística
    - Regresión Lineal
2. Máquinas de vector soporte
3. Algoritmos de clustering
4. Redes neuronales

Más información sobre el PCA:

https://towardsdatascience.com/the-most-gentle-introduction-to-principal-component-analysis-9ffae371e93b#:~:text=From%20Wikipedia%2C%20PCA%20is%20a,find%20unmeasured%20%E2%80%9Clatent%20variables%E2%80%9D.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Sara E. Rodríguez.
</footer>